In [11]:
from PIL import Image
import numpy as np
import random
import os

In [12]:
def del_all_files(folder_path):
    # フォルダ内のすべてのファイルを走査
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            # ファイルの場合のみ削除（ディレクトリは除く）
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            # もしディレクトリも削除したい場合は、ここで os.rmdir(file_path) を使用
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

In [13]:
def add_random_noise(image):
    # PIL画像をNumPy配列に変換
    np_image = np.array(image)

    # ノイズを生成 (ガウシアンノイズなど)
    noise = np.random.normal(loc=0, scale=25, size=np_image.shape)

    # 生成したノイズを画像に追加
    noisy_image = np_image + noise

    # 画像の値を0から255の範囲に制限
    noisy_image_clipped = np.clip(noisy_image, 0, 255)

    # NumPy配列をPIL画像に戻す
    noisy_image_pil = Image.fromarray(np.uint8(noisy_image_clipped))
    return noisy_image_pil

In [ ]:
def create_dice1_img_YOLO(num_img=1000):
    for idx_dice in range(1, 7):
        # 読み込み
        image_path = f'./dice_samples/sample_dice1_{idx_dice}.png'
        original_image = Image.open(image_path).convert('RGBA')

        # ダウンスケーリング
        downscaled_size = (700, 700)
        downscaled_image = original_image.resize(downscaled_size)
        for idx_img in range(num_img):
            # 背景画像生成
            background_size = (2000, 2000)
            background = Image.new("RGB", background_size, (0, 0, 0))

            # ランダムな角度で画像を回転
            random_angle = random.randint(0, 360)
            rotated_image = downscaled_image.rotate(random_angle, expand=True)

            # 回転後の画像サイズを取得
            rotated_size = rotated_image.size

            # 回転後の画像が背景内に収まるように座標を制限する
            max_x_rotated = background_size[0] - rotated_size[0]
            max_y_rotated = background_size[1] - rotated_size[1]

            # 座標が負にならないようにする
            max_x_rotated = max(max_x_rotated, 0)
            max_y_rotated = max(max_y_rotated, 0)

            # 回転後の画像を配置するランダムな位置を決定
            random_x_rotated = random.randint(0, max_x_rotated)
            random_y_rotated = random.randint(0, max_y_rotated)

            # 新しい黒い背景画像を生成（前の画像は破棄）
            background_rotated = Image.new("RGB", background_size, (0, 0, 0))

            # 回転した画像を背景に合成
            background_rotated.paste(rotated_image, (random_x_rotated, random_y_rotated))

            # 合成された画像をグレースケールに変換
            grayscale_image_rotated = background_rotated.convert("L")

            bbox_center_x = random_x_rotated + rotated_size[0] / 2
            bbox_center_y = random_y_rotated + rotated_size[1] / 2
            rel_center_x = bbox_center_x / background_size[0]
            rel_center_y = bbox_center_y / background_size[1]
            rel_width = rotated_size[0] / background_size[0]
            rel_height = rotated_size[1] / background_size[1]

            # アノテーションのフォーマットと保存
            annotation = f"{idx_dice-1} {rel_center_x} {rel_center_y} {rel_width} {rel_height}\n" 
            
            if idx_img % 10 == 0:
                folder = "test"
            elif idx_img % 10 == 1:
                folder = "valid"
            else:
                folder = "train"

            # グレースケール画像のパス
            grayscale_image_rotated_high = grayscale_image_rotated.resize((80, 80))
            grayscale_image_high_path = f'./output/original_set/{folder}/images/1{num_img * (idx_dice - 1)+idx_img + 1}.png'
            grayscale_image_rotated.save(grayscale_image_high_path)
            annotation_path = f'./output/original_set/{folder}/labels/1{num_img * (idx_dice - 1)+idx_img + 1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)   

            grayscale_image_rotated_low = grayscale_image_rotated.resize((20, 20))
            grayscale_image_low_path = f'./output/original_set/{folder}/images/2{num_img * (idx_dice - 1)+idx_img + 1}.png'
            grayscale_image_rotated_low.save(grayscale_image_low_path)
            annotation_path = f'./output/original_set/{folder}/labels/2{num_img * (idx_dice - 1)+idx_img + 1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)   

            grayscale_image_rotated_upsize = grayscale_image_rotated.resize((20, 20)).resize((80, 80))
            grayscale_image_upsize_path = f'./output/original_set/{folder}/images/3{num_img * (idx_dice - 1)+idx_img + 1}.png'
            grayscale_image_rotated_upsize.save(grayscale_image_upsize_path)
            annotation_path = f'./output/original_set/{folder}/labels/3{num_img * (idx_dice - 1)+idx_img + 1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)   

            noisy_image_high = add_random_noise(grayscale_image_rotated_high)
            noisy_image_high_path = f'./output/original_set/{folder}/images/4{num_img * (idx_dice - 1)+idx_img + 1}.png'
            noisy_image_high.save(noisy_image_high_path)
            annotation_path = f'./output/original_set/{folder}/labels/4{num_img * (idx_dice - 1)+idx_img + 1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)   

            noisy_image_low = add_random_noise(grayscale_image_rotated_low)
            noisy_image_low_path = f'./output/original_set/{folder}/images/5{num_img * (idx_dice - 1)+idx_img + 1}.png'
            noisy_image_low.save(noisy_image_low_path)
            annotation_path = f'./output/original_set/{folder}/labels/5{num_img * (idx_dice - 1)+idx_img + 1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)   

            grayscale_image_rotated_upsize = noisy_image_low.resize((80, 80))
            grayscale_image_upsize_path = f'./output/original_set/{folder}/images/6{num_img * (idx_dice - 1)+idx_img + 1}.png'
            grayscale_image_rotated_upsize.save(grayscale_image_upsize_path)
            annotation_path = f'./output/original_set/{folder}/labels/6{num_img * (idx_dice - 1)+idx_img + 1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)   

In [10]:
def main(num_img=1000):
    for f1 in ["train", "test", "valid"]:
        for f2 in ["images", "labels"]:
            folder_path = f'/home/imonalc/data_competition/dice/make_dice_img/output/original_set/{f1}/{f2}'
            del_all_files(folder_path)
    create_dice1_img_YOLO(num_img)

In [3]:
if __name__ == '__main__':
    main(10000)

In [4]:
# python scripts/generate_meta_info_pairdata.py --input ../make_dice_img/output/high_resolution/dice1_img ../make_dice_img/output/low_resolution/dice1_img --meta_info datasets/DF2K/meta_info/meta_info_DIV2K_sub_pair.txt
# make_dice_img/output/dice1_img zl real-ESRGAN/datasets/DF2K niido
# train